# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [4]:
# names us
def df_names_us():
    df = pd.DataFrame()
    for year in range(1880,2021):
        temp_df = pd.read_csv(f"names/yob{year}.txt",header=None, names=['name', 'gender', 'births'])
        temp_df["year"]=year
        temp_df=temp_df.loc[:, ["year",'name', 'gender', 'births']]
        df=df.append(temp_df,ignore_index=True)
    return df

In [5]:
names_us=df_names_us()

In [6]:
names_us.dtypes

year       int64
name      object
gender    object
births     int64
dtype: object

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [9]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    df=pd.read_csv("nat2020.csv", sep=";",header=0, 
               names=['gender','name','year','births'])[["year",'name', 'gender', 'births']]
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df = df.dropna(subset=['year'])
    df['year'] = df['year'].astype("int64")
    mapping = pd.Series({2: "F", 1: "M"})
    df['gender'] = df['gender'].map(mapping)
    df = df.drop(df[df.name.str.len()<2].index)
    df = df.drop(df[df.name == '_PRENOMS_RARES'].index)
    df['name'] = df['name'].astype(str)
    df["name"] = df.name.str.title()
    df.sort_values(['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    return df

In [10]:
df_names_fr()

,year,name,gender,births
0,2009,Aadam,M,4
1,2014,Aadam,M,3
2,2016,Aadam,M,4
3,2017,Aadam,M,4
4,2018,Aadam,M,3
...,...,...,...,...
630402,2016,Zyneb,F,6
630403,2017,Zyneb,F,6
630404,2018,Zyneb,F,5
630405,2019,Zyneb,F,7


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [11]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20210929.csv',sep=";")
    df = df.drop([0,1,2,3,4])
    df = df.set_index('Titre :')
    df.index = pd.to_datetime(df.index, format ='%d/%m/%Y' )
    df = df.rename_axis('Date')
    df = df.rename(lambda x: x[-4:-1], axis='columns')
    df = df.applymap(str).applymap(lambda x: str(x.replace(',','.')))
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna()
    df = df[devises]
    return df

### Tests

In [12]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [13]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [14]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 11.070s

OK
